In [ ]:
import numpy as np
import tensorflow as tf
  
physical_devices = tf.config.list_physical_devices('GPU')
tf.config.experimental.set_memory_growth(physical_devices[0], True)

In [ ]:
class MoCoQueueOld:
    def __init__(self, max_queue_len=128):
        self.Q = []
        self.max_queue_len = max_queue_len
#         self.zsize = zsize
#         self.Q = tf.zeros((max_queue_len, zsize), dtype=tf.float32)

    def __len__(self):
        return self.Q.shape[0]

    def enqueue(self, z):
        self.Q.append(z)
        if len(self) > self.max_queue_len:
            self.dequeue()

    def dequeue(self):
        """ Remove the oldest item in queue """
        _ = self.Q.pop(0)

    def getqueue(self):
        return tf.concat(self.Q, axis=0)

In [ ]:
class MoCoQueueNew:
    def __init__(self, max_queue_len=4096, zsize=128):
        self.Q = []
        self.max_queue_len = max_queue_len
        self.zsize = zsize
        self.Q = tf.zeros((max_queue_len, zsize), dtype=tf.float32)

    def __len__(self):
        return self.Q.shape[0]

    def enqueue(self, z):
        #self.Q.append(z)
        n = z.shape[0]
        i = tf.range(self.max_queue_len) < n
        self.Q = self.Q[~i]
        self.Q = tf.concat([self.Q, z], axis=0)

#     def getqueue(self):
#         return tf.concat(self.Q, axis=0)

In [ ]:
zsize = 8
batchsize = 16

queue = MoCoQueueNew(max_queue_len=64, zsize=zsize)
print(queue.Q)

for i in range(8):
    z = tf.random.normal(shape=(batchsize, zsize))
    queue.enqueue(z)
    
print(queue.Q)

In [ ]:
x = tf.zeros((16,4))
n = 4
i = tf.range(16) < n
p = x[i]
x = x[~i]
x = tf.concat([x,tf.random.normal((n, 4))], axis=0)
x